# Import Data

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
df2017=pd.read_csv("/content/drive/Shareddrives/21-1 대전시 공모전/일별데이터/17년_일별.csv")
df2018=pd.read_csv("/content/drive/Shareddrives/21-1 대전시 공모전/일별데이터/18년_일별.csv")
df2019=pd.read_csv("/content/drive/Shareddrives/21-1 대전시 공모전/일별데이터/19년_일별.csv")
df2020=pd.read_csv("/content/drive/Shareddrives/21-1 대전시 공모전/일별데이터/20년_일별.csv")
df2021=pd.read_csv("/content/drive/Shareddrives/21-1 대전시 공모전/일별데이터/21년_일별.csv")

In [4]:
df=pd.concat([df2017,df2018,df2019,df2020,df2021])

In [5]:
df.head()

,날짜,역명,이용자 수
0,2017-01-01,판암,4426
1,2017-01-01,신흥,1998
2,2017-01-01,대동,4478
3,2017-01-01,대전,15911
4,2017-01-01,중앙로,10464


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34122 entries, 0 to 1979
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      34122 non-null  object
 1   역명      34122 non-null  object
 2   이용자 수   34122 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [6]:
df['날짜']=pd.to_datetime(df['날짜'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34122 entries, 0 to 1979
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      34122 non-null  datetime64[ns]
 1   역명      34122 non-null  object        
 2   이용자 수   34122 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.0+ MB


In [7]:
train1=df[(df['날짜']>='2017-01-01') & (df['날짜']<='2018-12-31')]

In [8]:
g1=['중구청','대전','신흥','판암']
g2=['중앙로','오룡','용문','서대전네거리']
g3=['반석','지족','노은','월드컵경기장','정부청사','시청']
g4=['현충원','구암','유성온천','갑천','월평','갈마','탄방','대동']

In [9]:
g1train=train1.loc[train1['역명'].isin(g1)]
g2train=train1.loc[train1['역명'].isin(g2)]
g3train=train1.loc[train1['역명'].isin(g3)]
g4train=train1.loc[train1['역명'].isin(g4)]

In [10]:
g1train.head()

,날짜,역명,이용자 수
0,2017-01-01,판암,4426
1,2017-01-01,신흥,1998
3,2017-01-01,대전,15911
5,2017-01-01,중구청,1718
22,2017-01-02,판암,8306


In [11]:
test1=df[(df['날짜']>='2019-01-01') & (df['날짜']<='2019-12-31')]
g1test=test1.loc[test1['역명'].isin(g1)]
g2test=test1.loc[test1['역명'].isin(g2)]
g3test=test1.loc[test1['역명'].isin(g3)]
g4test=test1.loc[test1['역명'].isin(g4)]

In [12]:
X=pd.read_csv("/content/drive/Shareddrives/21-1 대전시 공모전/일별.csv")

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1551 entries, 0 to 1550
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   날짜           1551 non-null   object 
 1   일우량          1551 non-null   float64
 2   휴일           1551 non-null   int64  
 3   장마기간         1095 non-null   float64
 4   일최심신적설량(cm)  1095 non-null   float64
 5   혼잡도          1551 non-null   float64
 6   폭염           1551 non-null   int64  
 7   평균기온(참조)     1551 non-null   float64
 8   최저기온(참조)     1551 non-null   float64
 9   최고기온(참조)     1551 non-null   float64
dtypes: float64(7), int64(2), object(1)
memory usage: 121.3+ KB


In [13]:
X['날짜']=pd.to_datetime(X['날짜'])

In [14]:
X_train1=X[(X['날짜']>='2017-01-01') & (X['날짜']<='2018-12-31')]
X_test1=X[(X['날짜']>='2019-01-01') & (X['날짜']<='2019-12-31')]

# Modeling

In [15]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor

In [16]:
from sklearn.metrics import mean_squared_error
def rmse(y, pred):
    return np.sqrt(mean_squared_error(y,pred))

In [19]:
xgb_model = xgb.XGBRegressor(n_estimators=500, learning_rate= 0.1, max_depth=4,random_state=42)

In [25]:
def xgb_predict(model, X_train_full, X_test,y_train_full, y_test):
    y_train_grouped = y_train_full.groupby('날짜').mean()
    y_test_grouped = y_test.groupby('날짜').mean()
    X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_grouped['이용자 수'],test_size=0.2, random_state=42)
    X_train = X_train.sort_values(by='날짜')
    X_train = X_train.drop(['날짜'], axis=1)
    X_val = X_val.sort_values(by='날짜')
    X_val = X_val.drop(['날짜'], axis=1)
    X_test = X_test.drop(['날짜'],axis=1)
    #model
    xgb_model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)], eval_metric = 'rmse', early_stopping_rounds=100)
    xgb_pred = xgb_model.predict(X_test)
    #결과 확인
    print(rmse(y_test_grouped['이용자 수'], xgb_pred))

In [26]:
xgb_predict(xgb_model, X_train1, X_test1, g1train, g1test)

[15:22:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:8875.38
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:8012.82
[2]	validation_0-rmse:7238.41
[3]	validation_0-rmse:6543.54
[4]	validation_0-rmse:5920.5
[5]	validation_0-rmse:5362.64
[6]	validation_0-rmse:4861.7
[7]	validation_0-rmse:4415.82
[8]	validation_0-rmse:4017.37
[9]	validation_0-rmse:3671.1
[10]	validation_0-rmse:3366.08
[11]	validation_0-rmse:3095.51
[12]	validation_0-rmse:2847.85
[13]	validation_0-rmse:2637.98
[14]	validation_0-rmse:2444.71
[15]	validation_0-rmse:2275.21
[16]	validation_0-rmse:2140.27
[17]	validation_0-rmse:2017.27
[18]	validation_0-rmse:1920.6
[19]	validation_0-rmse:1832.58
[20]	validation_0-rmse:1765.84
[21]	validation_0-rmse:1701.63
[22]	validation_0-rmse:1641.78
[23]	validation_0-rmse:1594.04
[24]	validation_0-rmse:1561.72
[25]	validation_0-rmse:1533.74
[26]	validation